In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import gradio as gr
from PIL import Image, ImageTk
from faker import Faker
fake = Faker()

# Basic Dataset erstellen

3 Kategorien: Hotels, Restaurants & Aktivitäten

In [ ]:
df_rest = pd.read_csv('01_Data\dataset_Google-Maps-Reviews-Restaurants_2023-03-26_09-43-08-320.csv')
df_act = pd.read_csv('01_Data\dataset_Google-Maps-Reviews-Activities_2023-03-26_11-08-15-435.csv')
df_hotel = pd.read_csv('01_Data\dataset_Google-Maps-Reviews-Hotels_2023-03-26_11-34-16-492.csv')

print('Restaurants (Shape): ', df_rest.shape)
print('Activities (Shape): ', df_act.shape)
print('Hotels (Shape): ', df_hotel.shape)

In [ ]:
columns_to_keep = [
    "stars",
    "publishedAtDate",
    "name",
    "text",
    "title",
    "totalScore",
    "likesCount",
    "isLocalGuide",
    "reviewId",
    "categoryName",
    "reviewImageUrls/0",
    "reviewImageUrls/1",
    "reviewImageUrls/2",
    "reviewImageUrls/3",
    "reviewImageUrls/4",
    "reviewImageUrls/5",
    "reviewImageUrls/6",
    "reviewImageUrls/7",
    "reviewImageUrls/8",
    "reviewImageUrls/9",
    "reviewImageUrls/10",
    "reviewUrl",
    "reviewerId",
    "reviewerNumberOfReviews",
    "reviewerPhotoUrl",
    "reviewerUrl",
    "reviewsCount",
    "scrapedAt",
    "state",
    "street",
    "temporarilyClosed",
    "url",
    "website"
]

In [ ]:
df_rest = df_rest.dropna(subset=["reviewImageUrls/0"])
df_rest = df_rest[columns_to_keep]
print(df_rest.shape)

In [ ]:
df_act = df_act.dropna(subset=["reviewImageUrls/0"])
df_act = df_act[columns_to_keep]
print(df_act.shape)

In [ ]:
df_hotel = df_hotel.dropna(subset=["reviewImageUrls/0"])
df_hotel = df_hotel[columns_to_keep]
print(df_hotel.shape)

In [ ]:
df = pd.concat([df_rest, df_act, df_hotel], ignore_index=True)
print(df.shape)

In [33]:
image_urls_cols = [
    "reviewImageUrls/0",
    "reviewImageUrls/1",
    "reviewImageUrls/2",
    "reviewImageUrls/3",
    "reviewImageUrls/4",
    "reviewImageUrls/5",
    "reviewImageUrls/6",
    "reviewImageUrls/7",
    "reviewImageUrls/8",
    "reviewImageUrls/9"
]
print(image_urls_cols)

for col in image_urls_cols:
    df[col + "_hr"] = df[col]
    df[col + "_hr"] = df[col + "_hr"].str.replace("=w150-h150-k-no-p", "=w1281-h1281-p-k-no")


print(df.shape)

for index, row in df.iterrows():
    print("row: ", index)
    print(row["reviewImageUrls/0"])
    print(row["reviewImageUrls/0_hr"])

['reviewImageUrls/0', 'reviewImageUrls/1', 'reviewImageUrls/2', 'reviewImageUrls/3', 'reviewImageUrls/4', 'reviewImageUrls/5', 'reviewImageUrls/6', 'reviewImageUrls/7', 'reviewImageUrls/8', 'reviewImageUrls/9']
(6869, 43)
row:  0
https://lh5.googleusercontent.com/p/AF1QipNgNO--eBeUWJYGsTIdYJtmXdZAeCWDxOuiHZGq=w150-h150-k-no-p
https://lh5.googleusercontent.com/p/AF1QipNgNO--eBeUWJYGsTIdYJtmXdZAeCWDxOuiHZGq=w1281-h1281-p-k-no
row:  1
https://lh5.googleusercontent.com/p/AF1QipPoay5kH4dthf5yOo_ZE7Ey6GsbSU162Qjleyr1=w150-h150-k-no-p
https://lh5.googleusercontent.com/p/AF1QipPoay5kH4dthf5yOo_ZE7Ey6GsbSU162Qjleyr1=w1281-h1281-p-k-no
row:  2
https://lh5.googleusercontent.com/p/AF1QipN8lcJlwGh24EFKsjv0b-CmKp4wF3V6MUdjDYeM=w150-h150-k-no-p
https://lh5.googleusercontent.com/p/AF1QipN8lcJlwGh24EFKsjv0b-CmKp4wF3V6MUdjDYeM=w1281-h1281-p-k-no
row:  3
https://lh5.googleusercontent.com/p/AF1QipPhH5IUv8iqBYklKApwZFLzj0YMAWRITwkgx-vo=w150-h150-k-no-p
https://lh5.googleusercontent.com/p/AF1QipPhH5IUv8iqBY

## Gradio Anwendung Versuch

### HTML Bewertung im Stil von Google Maps erstellen

In [47]:
def generate_review_html(stars, date, user, is_local_guide, text, user_image, review_images):
    full_stars = int(stars)
    half_star = (stars % 1) > 0
    empty_stars = 5 - full_stars - half_star
    full_star_html = '<span class="star full">&#9733;</span>'
    half_star_html = '<span class="star half">&#9733;</span>'
    empty_star_html = '<span class="star empty">&#9733;</span>'

    stars_html = (full_star_html * full_stars) + (half_star_html * half_star) + (empty_star_html * empty_stars)
    
    user_name = user if not is_local_guide else f"{user} (local guide)"
    
    html = f"""
    <style>
      .star.full {{
        color: #f1c40f;
      }}
      .star.half {{
        color: #f1c40f;
        position: relative;
      }}
      .star.half::before {{
        content: '\\2605';
        position: absolute;
        left: 0;
        color: #ddd;
      }}
      .star.empty {{
        color: #ddd;
      }}
    </style>
    <div class="review">
      <div class="user">
        <img src={user_image} alt="User profile picture">
        <div class="user-info">
          <h4>{user_name}</h4>
          <p>Submitted on {date}</p>
        </div>
      </div>
      <div class="rating">
        {stars_html}
      </div>
      <div class="review-text">
        <p>{text}</p>
      </div>
      <div class="review-image">
        <img src={review_images} alt="Restaurant image">
      </div>
    </div>
    """
    return html

In [49]:
def generate_deepfake(review_about, stars):
    text = fake.text()
    stars = fake.random_int(min=1, max=5, step=1)
    date = fake.date_between(start_date='-1y', end_date='today')
    user = fake.name()
    local_guide = fake.boolean(chance_of_getting_true=7)
    html_review = generate_review_html(stars, date, user, local_guide, text, user_image="https://via.placeholder.com/50/%22%20alt=/%22Restaurant%20image/", review_images = "https://lh5.googleusercontent.com/p/AF1QipPsYMpfUpwmH5ltkPNIeDbgPivvBstyJCea32vJ=w150-h150-k-no-p")
    return html_review

def verify_review(stars, date, user, local_guide, text, image):
    return "FAKE", "0.9"

In [50]:
with gr.Blocks() as demo:
    with gr.Tab ("Create Fake Review"):
        review_about = gr.Textbox(label="Review about")
        review_stars = gr.Slider(label="Stars", minimum=1, maximum=5, step=1)
        generate_btn = gr.Button("Generate Multi-Modal Review")
        html_output = gr.HTML(label="Review") 
        generate_btn.click(fn=generate_deepfake, inputs=[review_about, review_stars], outputs=[html_output])
    
    with gr.Tab ("Verify Review"):
        with gr.Row(): 
            input_stars = gr.Slider(label="Stars", minimum=1, maximum=5, step=1)
            input_date = gr.Textbox(label="Date", default="2023-01-01")
            input_user = gr.Textbox(label="User")
            input_local_guide = gr.Checkbox(label="Local Guide")
       
        input_text = gr.Textbox(label="Review Text")
        input_image = gr.Image(label="Review Image")
        verify_btn = gr.Button("Verify Review")
        output_text = gr.Textbox(label="Generated Text")
        output_score = gr.Textbox(label="Score")
        verify_btn.click(fn=verify_review, inputs=[input_stars, input_date, input_user, input_local_guide, input_text, input_image], outputs=[output_text, output_score])

demo.launch(debug=True)

c:\Users\michi\AppData\Local\Programs\Python\Python39\lib\site-packages\gradio\deprecation.py:43: UserWarning: You have unused kwarg parameters in Textbox, please remove them: {'default': '2023-01-01'}
  warnings.warn(


Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.


Keyboard interruption in main thread... closing server.
